## Window Functions 

- [a window function](https://www.postgresql.org/docs/9.1/tutorial-window.html) performs a calculation across a set of table rows that are somehow related to the current row.

- Kind of row based approach to aggregation, row by row display for each step of calculation 

- You can’t use window functions and standard aggregations in the same query. More specifically, you can’t include window functions in a GROUP BY clause

###  <u> Partition By and Order BY </u>

- Order BY defines the frequency of aggregation ( like one calculation for each time_at)  whereas Partition BY splits the table into groups ( like accounts ) 

- This is comparable to the type of calculation that can be done with an aggregate function. But unlike regular aggregate functions, use of a window function does not cause rows to become grouped into a single output row — the rows retain their separate identities

- Window function is able to access more than just the current row of the query result.


 **=> step1** 

```sql
SELECT occurred_at, 
       SUM(standard_qty) as running_total
FROM orders
GROUP BY occurred_at
ORDER BY 1
```


 **=> step2** 

```sql
SELECT DATE_TRUNC('month',occurred_at) as month,
       SUM(standard_qty) as running_total
FROM orders
GROUP BY 1
ORDER BY 1
```

 **=> step3** 

```sql
SELECT occurred_at,standard_qty,
       SUM(standard_qty) OVER ( ORDER BY  occurred_at )  as running_total
FROM orders
```

 **=> step4** 

```sql
SELECT occurred_at,standard_qty,
       DATE_TRUNC('month',occurred_at) AS month,
       SUM(standard_qty) OVER ( ORDER BY  occurred_at )  as running_total
FROM orders
```

 **=> step5** 

```sql
SELECT occurred_at,standard_qty,
       DATE_TRUNC('month',occurred_at) AS month,
       SUM(standard_qty) OVER (PARTITION BY DATE_TRUNC('month',occurred_at))  as running_total
FROM orders
```

 **=> step6** 
```sql
SELECT occurred_at,standard_qty,
       DATE_TRUNC('month',occurred_at) AS month,
       SUM(standard_qty) OVER (PARTITION BY DATE_TRUNC('month',occurred_at)  ORDER BY  occurred_at )  as running_total
FROM orders
```

  - More details on **`OVER`** and **`PARTITION BY`** at [link](https://blog.sqlauthority.com/2015/11/04/sql-server-what-is-the-over-clause-notes-from-the-field-101/)
  
  
  - More details about **`PARTITION BY`** at [link](https://stackoverflow.com/questions/561836/oracle-partition-by-keyword) 
  
  
  - Not every window function uses PARTITION BY; we can also use ORDER BY or no statement at all depending on the query we want to run
  
  
  - The ORDER BY clause is one of two clauses integral to window functions. The ORDER and PARTITION define what is referred to as the “window”—the ordered subset of data over which calculations are made. Removing ORDER BY just leaves an unordered partition; in our query's case, each column's value is simply an aggregation (e.g., sum, count, average, minimum, or maximum) of all the standard_qty values in its respective account_id.
  
  
  - The easiest way to think about this - leaving the ORDER BY out is equivalent to "ordering" in a way that all rows in the partition are "equal" to each other. Indeed, you can get the same effect by explicitly adding the ORDER BY clause like this: ORDER BY 0 (or "order by" any constant expression), or even, more emphatically, ORDER BY NULL.




 ###  <u> Row_Number & RANK </u>
 
 - no need to specify a variable within the parantheses
 - **ROW** displays the number of given rows within defined window
 - **RANK** is similar but provides same rank for the two lines with same ORDER BY clause. 
 - **DENSERANK** is same with RANK, but doesnt skip values after assigning several rows with the same rank
 

 **=> step1** 

```SQL
SELECT id, account_id,occurred_at,
       ROW_NUMBER() OVER (ORDER BY account_id) AS row_num
FROM orders
```

 **=> step2** 
```SQL
SELECT id, account_id,occurred_at,
       ROW_NUMBER() OVER (PARTITION BY account_id ORDER BY account_id) AS row_num
FROM orders
LIMIT 100
```

 **=> step3** 
```SQL
SELECT id, account_id,occurred_at,
       ROW_NUMBER() OVER (PARTITION BY account_id ORDER BY occurred_at) AS row_num
FROM orders
LIMIT 100
```

 **=> step4** 
```SQL
SELECT id, account_id,
       DATE_TRUNC('month',occurred_at) AS month ,
       ROW_NUMBER() OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS row_num
FROM orders
LIMIT 100
```

 **=> step5** 
```SQL
SELECT id, account_id,
       DATE_TRUNC('month',occurred_at) AS month ,
       RANK() OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS row_num
FROM orders
LIMIT 100
```

 ### <u> Aggregates  </u>

```SQL 
SELECT id,account_id, standard_qty,DATE_TRUNC('month', occurred_at) AS month,
  DENSE_RANK() OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS dense_rank,
  SUM(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS sum_std_qty,
  COUNT(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS count_std_qty,
  AVG(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS avg_std_qty,
  MIN(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS min_std_qty,
  MAX(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS max_std_qty
FROM orders
```


```SQL 
SELECT id,account_id, standard_qty,DATE_TRUNC('month', occurred_at) AS month,
  DENSE_RANK() OVER (PARTITION BY account_id ) AS dense_rank,
  SUM(standard_qty) OVER (PARTITION BY account_id ) AS sum_std_qty,
  COUNT(standard_qty) OVER (PARTITION BY account_id ) AS count_std_qty,
  AVG(standard_qty) OVER (PARTITION BY account_id ) AS avg_std_qty,
  MIN(standard_qty) OVER (PARTITION BY account_id ) AS min_std_qty,
  MAX(standard_qty) OVER (PARTITION BY account_id ) AS max_std_qty
FROM orders
```

 ### <u> Aliases </u>
 
 - Defined by using `WINDOW` clause, usually between WHERE and GROUP BY
 
```SQL 
SELECT id,account_id, standard_qty,DATE_TRUNC('month', occurred_at) AS month,
  DENSE_RANK() OVER main_window AS dense_rank,
  SUM(standard_qty) OVER main_window AS sum_std_qty,
  COUNT(standard_qty) OVER main_window AS count_std_qty,
  AVG(standard_qty) OVER main_window AS avg_std_qty,
  MIN(standard_qty) OVER main_window AS min_std_qty,
  MAX(standard_qty) OVER main_window AS max_std_qty
FROM orders
WINDOW main_window AS (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at))
``` 
 


 ### <u> Comparing a Row to Previous Row by LAG and LEAD </u>
 
 -**LAG** returns the value from a previous row to the current row in the table
 
 -**LEAD** returns the value from the row following the current row in the table.
 
```SQL
SELECT account_id,standard_sum,
       LAG(standard_sum) OVER (ORDER BY standard_sum) AS lag,
       LEAD (standard_sum) OVER (ORDER BY standard_sum) AS lead,
       standard_sum - LAG(standard_sum) OVER (ORDER BY standard_sum) AS lag_difference, 
       LEAD (standard_sum) OVER (ORDER BY standard_sum) - standard_sum AS lead_difference
FROM (SELECT account_id, SUM(standard_qty) as standard_sum
FROM orders
GROUP BY 1
ORDER BY 2) subtable
```


```SQL
SELECT account_id,standard_sum,
       LAG(standard_sum) OVER (ORDER BY standard_sum) AS lag,
       LEAD (standard_sum) OVER (ORDER BY standard_sum) AS lead,
       standard_sum - LAG(standard_sum) OVER (ORDER BY standard_sum) AS lag_difference, 
       LEAD (standard_sum) OVER (ORDER BY standard_sum) - standard_sum AS lead_difference
FROM (SELECT occured_at, total_amt_usd
FROM orders
GROUP BY 1
ORDER BY 2) subtable
```